### Importação dos dados

In [22]:
import pandas as pd

dados = pd.read_csv("data.csv").drop(columns=["Unnamed: 0"])
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


### Separação treino e teste

In [23]:
import numpy as np
from sklearn.model_selection import train_test_split

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))


Treinaremos com 7500 elementos e testaremos com 2500 elementos


### Modelagem e Teste

In [24]:
from sklearn.metrics import accuracy_score

#### Dummy

In [25]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

A acurácia do dummy stratified foi 58.00%


#### Decision Tree

In [26]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)


A acurácia foi 71.92%


### Validação Cruzada

In [27]:
from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3, return_train_score=False)

media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print(f"Precisão varia entre {(media - 2*desvio_padrao)*100:.2f}% e {(media + 2*desvio_padrao)*100:.2f}%")

Precisão varia entre 74.99% e 76.57%


### Validação Cruzada com aleatoridade


Importante para evitar situações de "azar" em que todos os dados com uma única classificação.

In [28]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 3, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)

media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print(f"Precisão varia entre {(media - 2*desvio_padrao)*100:.2f}% e {(media + 2*desvio_padrao)*100:.2f}%")

Precisão varia entre 75.65% e 75.91%


### Gerando coluna "modelo" para agrupar


Levando em consideração que um modelo é uma característica muito importante para definir se um carro foi comprado ou não, testar o agrupando os modelos é uma boa forma de validação

In [29]:
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,21
2,89627.50,0,12,11440.79806,16
3,95276.14,0,3,43167.32682,3
4,117384.68,1,4,12770.11290,6


### Validação Cruzada em grupos

In [30]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 3)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, groups=dados.modelo, return_train_score=False)

media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print(f"Precisão varia entre {(media - 2*desvio_padrao)*100:.2f}% e {(media + 2*desvio_padrao)*100:.2f}%")

Precisão varia entre 72.91% e 78.66%


### Pipeline e Validação Cruzada com StandardScaler

In [31]:

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao',scaler), ('estimador',modelo)])

cv = GroupKFold(n_splits = 10)
results = cross_validate(pipeline, x, y, cv = cv, groups = dados.modelo, return_train_score=False)

media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print(f"Precisão varia entre {(media - 2*desvio_padrao)*100:.2f}% e {(media + 2*desvio_padrao)*100:.2f}%")

Precisão varia entre 74.54% e 78.72%
